<a href="https://colab.research.google.com/github/simonebugo/Big_Data/blob/main/Esercizio_in_aula7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Caricare il dataset, eliminare eventuali attributi inutili (giustificare la scelta), eliminare eventuali istanze duplicate,
eliminare le istanze che contengono valori nulli, trasformare opportunamente i valori categorici (consiglio: usare Label
Encoder) e dividere il dataset in train (3/4 del dataset) e test (1/4). Calcolare e valutare le predizioni di un
RandomForestRegressor. Effettuare alcune considerazioni sui risultati ottenuti, calcolando la metrica RMSE.

In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("dataset.csv")
df.head() #-> il target è il price

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [5]:
df.shape

(205, 26)

In [6]:
df.isnull().sum() #non ho colonne con valori nulli

,0
car_ID,0
symboling,0
CarName,0
fueltype,0
aspiration,0
doornumber,0
carbody,0
drivewheel,0
enginelocation,0
wheelbase,0


In [12]:
df.dtypes

,0
car_ID,int64
symboling,int64
CarName,object
fueltype,object
aspiration,object
doornumber,object
carbody,object
drivewheel,object
enginelocation,object
wheelbase,float64


In [13]:
#occorre eliminare l'id dell'auto siccome è una feature totalmente inutile
df.drop(['car_ID'],axis=1,inplace=True)

In [8]:
#elimino eventuali istanze duplicate
df = df.drop_duplicates()

In [9]:
#elimino eventuali istanze nulle
df = df.dropna()

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [18]:
le = LabelEncoder()
for column in df:
  if df[column].dtype == object:
    df[column] = le.fit_transform(df[column])
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('price', axis=1), df['price'], test_size=0.25, random_state=42)


In [21]:
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [26]:
#def di un dizionario con i modelli da testare è una convenzione che si fa, in quest caso il modello è uno solo
models = {'RandomForestRegressor' : RandomForestRegressor()}
def test_model(name, model, traindata, testdata, trainlabels, testlabels):
  model.fit(traindata, trainlabels)
  trainpred = model.predict(traindata)
  testpred = model.predict(testdata)
  print(f" ===== {name} =====")
  print(f"RMSE on train {root_mean_squared_error(trainlabels,trainpred)}")
  print(f"RMSE on test {root_mean_squared_error(testlabels,testpred)}")

for name, model in models.items():
  test_model(name, model, X_train, X_test, Y_train, Y_test)

 ===== RandomForestRegressor =====
RMSE on train 868.340367933287
RMSE on test 1942.2565145821022


Creare una pipeline in cui, a partire dal dataset utilizzato al punto precedente, i valori degli attributi carlength,
carwidth e carheight sono discretizzati in 5 intervalli, citympg e highwaympg sono trasformati con uno
StandardScaler e tutti gli altri attributi sono lasciati invariati.

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler

In [38]:
col_tran = ColumnTransformer(
    transformers=[
        ('discr', KBinsDiscretizer(n_bins = 5), ["carlength","carwidth","carwidth"]),
        ('std', StandardScaler(),["citympg","highwaympg"])
        ],
    remainder='passthrough'#per lasciare invariati gli altri attributi
)
pipeline = Pipeline(steps=[
                              ('col_tran', col_tran),
                              ('model', RandomForestRegressor()),
                             ])
test_model("Pipeline",pipeline,X_train,X_test,Y_train,Y_test) #riutilizzo funzione vista prima che fa feat predict e calcola RSE

 ===== Pipeline =====
RMSE on train 937.5214980216599
RMSE on test 1974.5201789479083


In [39]:
pipeline

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('col_tran',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('discr', KBinsDiscretizer(),
                                                  ['carlength', 'carwidth',
                                                   'carwidth']),
                                                 ('std', StandardScaler(),
                                                  ['citympg', 'highwaympg'])])),
                ('model', RandomForestRegressor())])

Creare una nuova pipeline che applica la SelectKBest al dataset utilizzato al punto 1 e aggiunge le componenti
ottenute alle componenti della pipeline del punto precedente. Valutare i valori migliori di k di SelectKBest, del numero
di intervalli in cui discretizzare carlength, carwidth e carheight e dei parametri criterion e max_depth del
RandomForestRegressor. Ignorare eventuali warning. Confrontare i risultati con quelli ottenuti precedentemente.

In [35]:
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import FeatureUnion

In [40]:
combined_features = FeatureUnion([("Kbest",SelectKBest()),("Coltran",col_tran)])
my_pipeline = Pipeline(steps=[
    ('combined_features', combined_features),
    ('estimator',RandomForestRegressor())
                              ],
                       verbose = True)
my_pipeline

Pipeline(steps=[('combined_features',
                 FeatureUnion(transformer_list=[('Kbest', SelectKBest()),
                                                ('Coltran',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('discr',
                                                                                  KBinsDiscretizer(),
                                                                                  ['carlength',
                                                                                   'carwidth',
                                                                                   'carwidth']),
                                                                                 ('std',
                                                                                  StandardScaler(),
                                                                                  ['citympg',
                                                                                   'highwaympg'])]))])),
                ('estimator', RandomForestRegressor())],
         verbose=True)

In [41]:
parameters = {
    'combined_features__Kbest__k': [2,4,6],
    'combined_features__Coltran__discr__n_bins': [5,10],
    'estimator__criterion': ["squared_error", "absolute_error", "friedman_mse", "poisson"],
    'estimator__max_depth':[7,10]
}

gd = GridSearchCV(my_pipeline, parameters)
gd.fit(X_train,Y_train)
Y_pred = gd.predict(X_test)
print(f"RMSE {root_mean_squared_error(Y_test,Y_pred)}")
print(f"bestparam {gd.best_.best_params_}")

/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s
[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.2s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.2s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


...

[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.7s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


[Pipeline] ......... (step 2 of 2) Processing estimator, total=   0.6s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


KeyboardInterrupt: 

Creare una pipeline che, a partire dal dataset originale, trasforma le colonne testuali in valori numerici e le feature
numeriche attraverso lo StandardScaler e applica il RandomForestRegressor. Come variano le performance?

In [42]:
df = pd.read_csv("dataset.csv") #vuole il dataset non modificato
categorical_features = []
numeric_features = []

for column in df.columns:
    if df[column].dtype == object:
        categorical_features.append(column)
    else:
        numeric_features.append(column)

numeric_features.remove('car_ID') #feature che avevamo detto essre inutile
numeric_features.remove('price') #è il target

In [43]:
from sklearn.preprocessing import OneHotEncoder
#per trasfrormare le colonne testuali in valori numerici si usa il one hot encoder, con l'ignore in modo da gestire i valori sconosciuti
coltran = ColumnTransformer(transformers=[("onehot", OneHotEncoder(handle_unknown='ignore'), categorical_features),
                                          ("std", StandardScaler(), numeric_features)],
                                 remainder='passthrough')

pipeline = Pipeline(steps=[("coltran", coltran),
                           ("estimator", RandomForestRegressor())])
test_model("Pipeline", pipeline, X_train, X_test, Y_train, Y_test)

 ===== Pipeline =====
RMSE on train 922.2451808839803
RMSE on test 1996.8427770165222
